In [2]:
import os
import pandas as pd
from pandas import read_csv, read_excel, DataFrame
from sae import COTSCar
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np
from typing import List

import sys
sys.path.append('../')
import saedfsc

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Price of the car.

In [3]:
maxProductPrice = 400000

Information about customers

In [4]:
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_partworth_dict = saedfsc.customers.set_index('Name')['PartworthUtilityWeights'].to_dict()
name_weights_perf_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWtsPartworth = {c : np.fromstring(name_weights_partworth_dict[c].strip('[]'), sep=',') for c in name_weights_partworth_dict}
cWtsPerf = {c : np.fromstring(name_weights_perf_dict[c].strip('[]'), sep=',') for c in name_weights_perf_dict}

Methods to help compute the market share for each customer type (based on logit model).

In [5]:
def getTotalUtilityForCustomer(car : COTSCar, c : str, utilFn = 'partworth'):
    total_utility = 0
    pricePerf = (maxProductPrice - car.price) / maxProductPrice # Normalized price (0 is best, 1 is worst)
    if utilFn == 'partworth':
        product_utility = car.partworth_objectives(weights=cWtsPartworth[c])[0]
        total_utility += (1-cPriceFocus[c])*product_utility - cPriceFocus[c]*pricePerf
    elif utilFn == 'performance':
        product_utility = car.objectives(weights=cWtsPerf[c])[0]
        total_utility += (1-cPriceFocus[c])*product_utility + cPriceFocus[c]*pricePerf
    return total_utility

def getMarketShare(car : COTSCar, c : str, competitors : List[COTSCar], utilFn = 'partworth'): # based on logit model of demand
    carUtility = getTotalUtilityForCustomer(car, c, utilFn)
    totalCompetitorUtility = sum([getTotalUtilityForCustomer(competitorCar, c, utilFn) for competitorCar in competitors])
    return carUtility / (totalCompetitorUtility + carUtility)

Create a set of competitors.

In [7]:
profitMargin = 0.1
competitors = []
for i in range(3):
    competitorCar = COTSCar()
    competitorCar.price = (1+profitMargin)*competitorCar.cost()
    competitorCar.name = "Competitor Car " + str(i)
    competitors.append(competitorCar)

names = [car.name for car in competitors]
designs = [car.vector for car in competitors]
salesPrices = ['$' + format(car.price, ",.2f") for car in competitors]
costs = ['$' + format(car.cost(), ",.2f") for car in competitors]
competitorsDF = pd.DataFrame({
    'Name': names,
    'Design': designs,
    'SalesPrice': salesPrices,
    'ProductCost': costs
})
competitorsDF

,Name,Design,SalesPrice,ProductCost
0,Competitor Car 0,"[10, 0, 9, 4, 10, 4, 6, 17, 8, 2, 62, 64, 1, 0...","$121,464.95","$110,422.68"
1,Competitor Car 1,"[9, 9, 10, 3, 5, 4, 1, 9, 21, 2, 3, 83, 4, 0, ...","$6,803.74","$6,185.22"
2,Competitor Car 2,"[5, 1, 0, 7, 2, 3, 5, 18, 12, 2, 26, 164, 0, 3...","$5,220.15","$4,745.59"


## Partworth Utility

Create a random car and compute market share

In [8]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
product.vector = carDesignVec
product.price = (1 + profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price: $", format(product.price, ",.2f"))
print("Cost: $", format(product.cost(), ",.2f"))

qtys = [cQty[c] for c in customers]
utilities = [getTotalUtilityForCustomer(product, c) for c in customers]
market_shares = [getMarketShare(product, c, competitors) for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors)*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Quantity': qtys,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})
total_profit = product.price*df['Expected demand'].sum()
print("Total expected profit: $", format(total_profit, ",.2f"))
print("Market information for our car:")
df

OUR CAR:
Design:  [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
Sales price: $ 286,626.73
Cost: $ 260,569.75
Total expected profit: $ 449,307,584.75
Market information for our car:


,Name,Quantity,Utility for our car,Market share,Expected demand
0,CustomerType1,469,-0.308284,0.104300,48.916587
1,CustomerType2,929,-0.301814,0.101146,93.964593
2,CustomerType3,294,-0.380129,0.126671,37.241258
3,CustomerType4,834,-0.207723,0.092582,77.213261
4,CustomerType5,733,-0.404133,0.115312,84.523979
5,CustomerType6,35,-0.666431,0.170634,5.972182
6,CustomerType7,239,-0.824332,0.162255,38.778872
7,CustomerType8,564,-0.465186,0.133043,75.036467
8,CustomerType9,927,-0.533958,0.133360,123.624910
9,CustomerType10,366,-0.284829,0.105932,38.771026


## Performance Utility

In [9]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
print("COTSCar vec:", product.get_vec())
print("Car vec:", product.car.get_vec())
print("Cost: $", format(product.cost(), ",.2f"))
product.set_vec(carDesignVec)
print("COTSCar vec:", product.get_vec())
print("Car vec:", product.car.get_vec())
print("Cost: $", format(product.cost(), ",.2f"))
product.price = (1 + profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price: $", format(product.price, ",.2f"))
print("Cost: $", format(product.cost(), ",.2f"))

qtys = [cQty[c] for c in customers]
utilities = [getTotalUtilityForCustomer(product, c, 'performance') for c in customers]
market_shares = [getMarketShare(product, c, competitors, 'performance') for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors, 'performance')*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Quantity': qtys,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})
total_profit = product.price*df['Expected demand'].sum()
print("Total expected profit: $", format(total_profit, ",.2f"))
print("Market information for our car:")
df

COTSCar vec: [10, 4, 2, 6, 2, 3, 1, 0, 33, 4, 59, 184, 1, 1, 212, 194, 74]
Car vec: [0.3625, 0.15, 0.3926990815, 0.1125, 0.275, 1.65, 0.3926990815, 0.1125, 0.30000000000000004, 0.225, 0.3926990815, 0.894, 0.894, 0.9190748737045349, 2.2510355338329884, 1.0527957846008364, 0.002023418758763538, 0.2582733483154878, 0.31074947907029365, 10, 0, 4, 8, 10, 1, 6, 3, 16, 4, 0.9825304023992567, 0.9644484050228862, 0.14736759889409623, 0.1404283265640983, 0.2639568547227884, 0.5406223798297347, 0.514182021245328, 0.29789687345759275, 0.28337273877708935, 0.2879200773259199]
Cost: $ 472,532.57
COTSCar vec: [10, 4, 2, 6, 2, 3, 1, 0, 33, 4, 59, 184, 1, 1, 212, 194, 74]
Car vec: [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4, 0.2582733483154878, 0.31074947907029365, 10, 0, 4, 8, 10, 1, 6, 3, 16, 4, 0.9825304023992567, 0.9644484050228862, 0.14736759889409623, 0.1404283265640983, 0.2639568547227884, 0.5406223798297347, 0.514182021245328, 0.29789687345759275, 0.28337273877708935, 0.2879200773

,Name,Quantity,Utility for our car,Market share,Expected demand
0,CustomerType1,469,-984.954411,1.006099,471.860290
1,CustomerType2,929,-279.864340,1.010500,938.754602
2,CustomerType3,294,50.640782,0.922558,271.231950
3,CustomerType4,834,18.326123,0.804971,671.345767
4,CustomerType5,733,-378.240048,1.010150,740.439682
5,CustomerType6,35,-710.256270,1.009006,35.315223
6,CustomerType7,239,-579.696618,1.006085,240.454305
7,CustomerType8,564,-487.852175,1.006787,567.827601
8,CustomerType9,927,-392.319977,1.011614,937.766093
9,CustomerType10,366,155.083923,0.966964,353.908965
